## Текст задания

Задание: "Проанализировать трейдинговую активность наших юзеров".

Комментарий: Финтех компания разрабатывает и поддерживает свою собственную трейдинговую платформу. Данные "trading_data_copy.csv" в приложении.

Логику ответа и анализируемые данные можно определить самостоятельно, но хотелось бы получить ответы на следующие вопросы:

- Какое торговое поведение свойственно юзерам по регионам и платформам? В чем сходства и различия?
- Исходя из общих поведенческих паттернов, на какие сегменты можно разбить юзеров?
- Проанализировать данные, написать выводы и дать ответ на вопрос Топ-менеджменту “Как прошли последние 3 месяца деятельности компании и в частности какие результаты за Июнь?”. 

Комментарии по датасету: PnL - доход юзера с инструмента

## Импортируем библиотеки для анализа, а также загрузим данные

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import warnings


from dateutil.relativedelta import relativedelta

%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
!gdown 1_Qbh77K5LL6P6T9ISuyrBUbT8EkyaftK

Downloading...
From: https://drive.google.com/uc?id=1_Qbh77K5LL6P6T9ISuyrBUbT8EkyaftK
To: /content/trading_data_copy.csv
100% 29.6M/29.6M [00:00<00:00, 72.2MB/s]


In [ ]:
trading_data = pd.read_csv("trading_data_copy.csv")
trading_data

,user_id,user_registration_time,deal_create_time,deal_close_time,instrument_type,asset,deal_platform,region,pnl
0,511818141,2020-05-04 01:29:02.035977,2020-05-21 08:24:27.21+03,2020-05-21 08:25:03.66+03,digital-option,AUDJPY,Android,America (LATAM),-1.666610
1,144808927,2017-06-06 03:45:53.000000,2020-05-21 08:20:07.272+03,2020-05-21 08:25:06.323+03,digital-option,EURUSD,Web,ACO,-9.599913
2,518331028,2020-05-11 21:19:02.779547,2020-05-22 06:01:04.244+03,2020-05-22 06:02:02.791+03,digital-option,GBPUSD,Web,America (LATAM),-5.863622
3,483739956,2020-04-04 07:39:54.644213,2020-05-21 08:20:21.18+03,2020-05-21 08:25:04.869+03,digital-option,USDCHF,Android,ACO,-1.666620
4,483739956,2020-04-04 07:39:54.644213,2020-05-21 08:21:18.218+03,2020-05-21 08:25:06.067+03,digital-option,AUDCAD,Android,ACO,-1.666665
...,...,...,...,...,...,...,...,...,...
199995,341599312,2019-03-21 07:14:21.000000,2020-05-24 08:45:03.843+03,2020-05-24 08:49:59.169+03,digital-option,USDJPY-OTC,Web,America (LATAM),-0.168462
199996,184441270,2017-10-16 02:26:03.000000,2020-05-22 06:05:20.253+03,2020-05-22 06:06:02.219+03,digital-option,EURUSD,Web,America (LATAM),1.402720
199997,300965817,2018-09-06 02:56:07.000000,2020-05-13 08:19:40.286+03,2020-05-13 08:21:02.731+03,digital-option,USDCHF,Web,America (LATAM),25.263702
199998,472410662,2020-03-22 00:47:29.723404,2020-05-13 08:20:01.198+03,2020-05-13 08:21:02.67+03,digital-option,USDCHF,Web,America (LATAM),-1.154300


## Рассмотрим торговое поведение юзеров

### Построение модели

Под торговым поведением будем понимать среднее количество сделок за 1 день, средний доход пользователя от одной сделки и общий доход по региону и платформе. 

In [ ]:
DATE_FORMAT = "%Y-%m-%d"


avg_deal_counts = []

def get_avg_deal_count(data: pd.DataFrame) -> None:
    first_create_time = dt.datetime.strptime(data.deal_create_time.min().split()[0], DATE_FORMAT).date()
    last_close_time = dt.datetime.strptime(data.deal_close_time.max().split()[0], DATE_FORMAT).date()
    trading_days = (last_close_time - first_create_time).days
    avg_deal_counts.append(data.count()[0] / trading_days)

trading_data = pd.read_csv('trading_data_copy.csv')
trading_data_aggregated = trading_data.groupby(by=["region", "deal_platform"]).pnl.sum().to_frame()
trading_data_aggregated.rename(columns={"pnl": "pnl_sum"}, inplace=True)
trading_data_aggregated["user_amount"] = trading_data.groupby(by=["region", "deal_platform"]).user_id.nunique()
trading_data_aggregated["pnl_per_user"] = (trading_data_aggregated["pnl_sum"] /
                                            trading_data_aggregated["user_amount"])
trading_data.groupby(by=["region", "deal_platform"]).apply(get_avg_deal_count)
trading_data_aggregated["avg_deal_count"] = avg_deal_counts
trading_data_aggregated

pnl_sum  user_amount  pnl_per_user  \
region          deal_platform                                            
ACO             Android       -29563.019070         8716     -3.391810   
                Web            14235.462532         6010      2.368629   
                Web_mobile      1325.666003           13    101.974308   
                iOS           -18094.089377         2430     -7.446127   
Africa          Android        -5906.852510         2799     -2.110344   
                Web            -5854.592908         1681     -3.482804   
                Web_mobile       -80.744570            2    -40.372285   
                iOS             1587.501658          484      3.279962   
America (LATAM) Android       -16304.791672        19736     -0.826145   
                Web           -42123.417917        38306     -1.099656   
                Web_mobile        10.407873           38      0.273891   
                iOS           -14330.548558         5155     -2.779932   
China           Android          595.475652           45     13.232792   
                Web              546.100608          151      3.616560   
                iOS            -1775.726378           27    -65.767644   
Non-reg Europe  Android        -3147.245373          132    -23.842768   
                Web              202.067032          157      1.287051   
                Web_mobile        13.318977            1     13.318977   
                iOS            -2031.994582           21    -96.761647   
Reg Europe      Android        -4989.993162          482    -10.352683   
                Web           -33943.447568          835    -40.650835   
                iOS            -6242.111632          233    -26.790179   

                               avg_deal_count  
region          deal_platform                  
ACO             Android             32.103763  
                Web                 22.857995  
                Web_mobile           1.000000  
                iOS                 10.073626  
Africa          Android            113.612903  
                Web                 68.049180  
                Web_mobile           0.133333  
                iOS                 38.388889  
America (LATAM) Android             40.127473  
                Web                 86.174431  
                Web_mobile           1.642857  
                iOS                 10.717045  
China           Android             13.903226  
                Web                 19.500000  
                iOS                  6.366667  
Non-reg Europe  Android              0.447174  
                Web                 13.478261  
                Web_mobile           0.300000  
                iOS                  0.148768  
Reg Europe      Android              0.928894  
                Web                  1.802775  
                iOS                  0.377919

### Анализ полученных результатов

#### Самые лучшие и худшие пары регионов и платформ по каждому из столбцов

In [ ]:
for column in trading_data_aggregated.columns:
    best_in_column = trading_data_aggregated.nlargest(1, column)
    worst_in_column = trading_data_aggregated.nsmallest(1, column)
    print("\nColumn:", column)
    print("Best:", best_in_column[column][0], best_in_column.index[0])
    print("Worst:", worst_in_column[column][0], worst_in_column.index[0])


Column: pnl_sum
Best: 14235.462531666668 ('ACO', 'Web')
Worst: -42123.417916666665 ('America (LATAM)', 'Web')

Column: user_amount
Best: 38306 ('America (LATAM)', 'Web')
Worst: 1 ('Non-reg Europe', 'Web_mobile')

Column: pnl_per_user
Best: 101.97430794871795 ('ACO', 'Web_mobile')
Worst: -96.76164674603176 ('Non-reg Europe', 'iOS')

Column: avg_deal_count
Best: 113.61290322580645 ('Africa', 'Android')
Worst: 0.13333333333333333 ('Africa', 'Web_mobile')


#### По результатам анализа можно заключить:

Сходства:


*   Во многих регионах и платформах средний заработок пользователя и общий результат по всем транзакция отрицательный, кроме платформы Web-Mobile - на ней по всем регионам наблюдается прибыль.
*   Самой непопулярной платформой во всех регионах является Mobile Web (минимальное количество пользователей также принадлежит этой платформе в Non-Reg Europe с показателем 1 пользователь)
*   В большинстве регионов наиболее популярной платформой является Web (минимальное количество пользователей также принадлежит этой платформе в America (LATAM) с показателем 38306 пользователей)
*   В большинстве регионов платформа iOS является самой убыточной в пересчёте на пользователя.

Различия:


*   Регион Africa по распределению пользователей довольно сильно отличается от остальных: самая популярная платформа Android, самая убыточная на пользователя - Mobile Web.
*   Заметна значительная разница между общим количеством пользователей в регионах. Так, в China всего 223 уникальных аккаунта, в то время, как, к примеру, в Africa их ~5000, хотя техническое развитие и общее количество населения этих регионов несопоставимы.




## Паттерны поведения

### Обработка данных

В своей работе я реализовал 4 базовых паттерна поведения на основе 2 критериев: среднее количество торговых операций в день и средний доход от операции по сравнению со средним в регионе и на платформе пользователя. Семантическое название классов отвечает его контенту.

In [ ]:
behavior_data = trading_data.copy(deep=True)
last_close_time = dt.datetime.strptime(behavior_data.deal_close_time.max().split()[0], DATE_FORMAT).date()
behavior_data.user_registration_time = behavior_data.user_registration_time.apply(lambda x:
                                                                                  dt.datetime.strptime(x.split()[0],
                                                                                    DATE_FORMAT).date())

behavior_data["days_trading"] = (last_close_time - behavior_data.user_registration_time).dt.days
trade_avg = behavior_data.groupby(by=["user_id", "days_trading"]).size().to_frame()
trade_avg.rename(columns={0: "deals_per_day"}, inplace=True)
trade_avg["deals_per_day"] = trade_avg["deals_per_day"] / trade_avg.index.get_level_values(1)
trade_avg = trade_avg.droplevel("days_trading")
pnl_avg = behavior_data.groupby(["user_id"]).pnl.mean().to_frame()
pnl_avg.rename(columns={"pnl": "pnl_avg"}, inplace=True)
average_characteristics = trade_avg.join(pnl_avg)

behavior_data["active_loser"] = 0
behavior_data["active_winner"] = 0
behavior_data["passive_loser"] = 0
behavior_data["passive_winner"] = 0

average_characteristics

,deals_per_day,pnl_avg
user_id,,
2070,0.000828,-14.716585
337946,0.007328,-0.062908
360923,0.006903,0.237190
449611,0.003898,-0.633779
818646,0.003518,-0.665668
...,...,...
540147279,0.500000,-0.029528
540155958,0.500000,0.568262
540172840,1.000000,1.402543


In [ ]:
def check_user_score_and_activity(user: pd.Series, 
                                  average_characteristics: pd.DataFrame, 
                                  trading_data_aggregated: pd.DataFrame) -> pd.Series:
    user_id = user.user_id
    user_region, user_platform = user.region, user.deal_platform
    user_avg_deal, user_avg_pnl = average_characteristics.loc[user_id].values
    region_avg_pnl, region_avg_deal = trading_data_aggregated.loc[user_region, user_platform][[
        "pnl_per_user", "avg_deal_count"]]

    if user_avg_pnl > region_avg_pnl:
        if user_avg_deal > region_avg_deal:
            user["active_winner"] = 1
        else:
            user["passive_winner"] = 1
    else:
        if user_avg_deal > region_avg_deal:
            user["active_loser"] = 1
        else:
            user["passive_loser"] = 1

    return user

behavior_data = behavior_data.apply(check_user_score_and_activity, 
                                    axis=1, 
                                    args=(average_characteristics, trading_data_aggregated))

In [ ]:
behavior_df = {}
passive_loser_count = behavior_data.passive_loser.sum()
passive_winner_count = behavior_data.passive_winner.sum()
active_loser_count = behavior_data.active_loser.sum()
active_winner_count = behavior_data.active_winner.sum()
count = [passive_loser_count, passive_winner_count, active_loser_count, active_winner_count]
print(f"Passive losers = {passive_loser_count}\n"
      f"Passive winners = {passive_winner_count}\n"
      f"Active losers = {active_loser_count}\n"
      f"Active winners = {active_winner_count}")
behavior_df["count"] = count

Passive losers = 54419
Passive winners = 145495
Active losers = 0
Active winners = 86


In [ ]:
passive_loser_mean = behavior_data[behavior_data.passive_loser == 1].pnl.mean()
passive_winner_mean = behavior_data[behavior_data.passive_winner == 1].pnl.mean()
active_loser_mean = behavior_data[behavior_data.active_loser == 1].pnl.mean()
active_winner_mean = behavior_data[behavior_data.active_winner == 1].pnl.mean()
mean = [passive_loser_mean, passive_winner_mean, active_loser_mean, active_winner_mean]
print(f"Passive losers mean = {passive_loser_mean}\n"
      f"Passive winners mean = {passive_winner_mean}\n"
      f"Active losers mean = {active_loser_mean}\n"
      f"Active winners mean = {active_winner_mean}")
behavior_df["mean"] = mean

Passive losers mean = -10.438005951199644
Passive winners mean = 2.7641336259894382
Active losers mean = nan
Active winners mean = -0.16687205426356586


In [ ]:
passive_loser_sum = behavior_data[behavior_data.passive_loser == 1].pnl.sum()
passive_winner_sum = behavior_data[behavior_data.passive_winner == 1].pnl.sum()
active_loser_sum = behavior_data[behavior_data.active_loser == 1].pnl.sum()
active_winner_sum = behavior_data[behavior_data.active_winner == 1].pnl.sum()
sum = [passive_loser_sum, passive_winner_sum, active_loser_sum, active_winner_sum]
behavior_df["sum"] = sum
print(f"Passive losers sum = {passive_loser_sum}\n"
      f"Passive winners sum = {passive_winner_sum}\n"
      f"Active losers sum = {active_loser_sum}\n"
      f"Active winners sum = {active_winner_sum}")

Passive losers sum = -568025.8458583334
Passive winners sum = 402167.6219133333
Active losers sum = 0.0
Active winners sum = -14.350996666666664


In [ ]:
 behavior_df = pd.DataFrame(behavior_df, index=["Passive losers", "Passive winners", "Active losers", "Active winners"])
 behavior_df

,count,mean,sum
Passive losers,54419,-10.438006,-568025.845858
Passive winners,145495,2.764134,402167.621913
Active losers,0,NaN,0.000000
Active winners,86,-0.166872,-14.350997


### Выводы по вопросу

Получили, что пользователей, количество ставок больше среднего по региону и платформе, но при этом прибыль меньше средней - нет. Самый распространённый класс - пассивный с доходом, таких пользователей около ~75%. К слову люди, относящиеся к этому классу, единственные, кто в среднем имеют прибыль. Прибыль таких пользователей окупается проигрышем пассивных пользователей с убытком, таких ~25%, общая сумма их проигрыша превышает доход всех остальных. Выборка класса активных пользователей с доходом выше среднего по платформе и региону слишком мала, выводы по ней будут нерепрезентативны.

## Последние 3 месяца

### Построение модели

В сводном отчёте я решил оценивать месяца по критериям: количество зарегистрировавшихся пользователей и суммарный оборот от сделок в конкретном месяце.

In [ ]:
all_dates = trading_data.user_registration_time.values +\
           trading_data.deal_create_time.values +\
           trading_data.deal_close_time.values
date_now = dt.datetime.strptime(max(list(map(lambda x: x.split()[0], all_dates))), DATE_FORMAT).date()
print(date_now)
print(f"Last month: {date_now.month}")

2020-06-10
Last month: 6


In [ ]:
months = ["April", "May", "June"]
pnl_sums = []
registered_users = []

for month in [4, 5, 6]:
    pnl_sums.append(trading_data[(f"2020-0{month}" < trading_data.deal_create_time)
                        & (trading_data.deal_create_time < f"2020-0{month + 1}")].pnl.sum())
    registered_users.append(trading_data[(f"2020-0{month}" < trading_data.user_registration_time)
                        & (trading_data.user_registration_time < f"2020-0{month + 1}")].count()[0])

months_data = pd.DataFrame({
    "month": months,
    "pnl_sum": pnl_sums,
    "registered users": registered_users
})

months_data

,month,pnl_sum,registered users
0,April,43040.891117,32548
1,May,-161238.873397,35571
2,June,-9032.263595,979


### Вывод по 3 месяцам

Полученная таблица является не совсем репрезентативной, ведь доступные данные окачиваются на дате 10-06-22, таким образом дать объективный анализ последнего месяца - Июня - проблематично. 

Если попробовать экстраполировать параметры, то можно заметить отрицательную динамику в количестве пользователей (~1000 за 10 дней при норме ~10000). В случае pnl_sum нельзя дать однозначного прогноза, однако можно предположить, что он несколько стабилизируется, т.к. новичков регистрируется значительно меньше, чем в прошлых 2 месяцах. Я думаю, что это связано с сезоном отпусков, в среднем человек не хочет проводить свободное время за торговлей.

Если сравнивать Апрель и Май, то можно увидеть положительную динамику в количестве зарегистрировавшихся пользователей (+9,4%), при этом сумма сильно снизилась. Возможно, это связано с тем, что прибыло много неопытных пользователей, благодаря рекламной кампании.